# Extract spectral-temporal metrics at point-locations from SHP-File
#### Input from Julian Oeser

### Current staus: calculates classification, and visualizes it. Next steps include:
* Limited to small scale of trianing data - solve through tiling the study area and do sampleRegion per tile (and merge the featurecollections afterwards)
* Do this for multiple years - "time-calibrated"
* make the export of the classifications automated per tile

In [1]:
import ee  # the Earth Engine Python API library
ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook
from baumiTools import baumiEE, baumiVT
import json
import ogr, osr
import os
import pandas as pd

## Load a point-shapefile, convert it into feature collection
here, we load a shapefile with the points (e.g., from training data) for which we want to extrac the spectral-temporal features from GEE.

In [2]:
shp = baumiVT.CopyToMem("C:/Miniconda3/envs/py37/Lib/site-packages/baumiTools/geeTools/_OtherSources/_Julian/training_data/training_points_testSub3.shp")
#tempFolder = "G:/Baumann/temp/"
field = "id"
clasLabel = "wetland"
yrField = "Year"

fc = baumiEE.Convert_SHP_to_FC(shp.GetLayer(), field, clasLabel)

In [3]:
# Remember: has to be named class because the function Convert_SHP_to_FC has this as the classification label
wetland = fc.filter(ee.Filter.eq('Class',1))
other = fc.filter(ee.Filter.eq('Class',0))

In [11]:
# test 228077
median_L8 = baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=1, endMonth=3, path=228, row=77, band_rename=False)#, roi=geom_EE)# additional arguments: roi=feat_EE or path=227, row=28
for win in [[4,6], [7,9],[10,12]]:
    median_L8 = median_L8.addBands(baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=win[0], endMonth=win[1], path=228, row=77))

In [5]:
#trainingSample = median_L8.sampleRegions(fc,["wetland"],100)
#trainingSample
trainingSample = median_L8.sampleRegions(
  collection = fc,
  properties = ['Class'],
  scale = 30,
  tileScale = 2  
)
#trainingSample.getInfo()

In [6]:
bandNames = median_L8.bandNames()
classifier = ee.Classifier.randomForest(100,0).setOutputMode('PROBABILITY').train(trainingSample,'Class',bandNames)
classification = median_L8.classify(classifier)#.multiply(100)
#bandNames.getInfo()

In [10]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

Map = ipyleaflet.Map(center=(-14.438411, -63.743799), zoom=4,
                      layout={'height':'600px'},
                      scroll_wheel_zoom=True)
classific = ipyleaflet.TileLayer(name="classification", url=GetTileLayerUrl(
    classification.visualize(min=0, max=1, gamma=1.5)))
Landsat8 = ipyleaflet.TileLayer(name="Landsat8", url=GetTileLayerUrl(
    median_L8.visualize(min=0, max=8000, gamma=1.5,bands= ['NIR_2019-01-03', 'SWIR1_2019-01-03', 'R_2019-01-03'])))

Map.add_layer(Landsat8)
Map.add_layer(classific)

Map.add_control(ipyleaflet.LayersControl(position='topright'))

Map

Map(center=[-14.438411, -63.743799], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…